# Nipype sandbox


Nipype basic doc

- [iterables](http://nipype.readthedocs.io/en/latest/users/mapnode_and_iterables.html#iterables)
- `help()` method to see function documentation.
- `input_spec()` method to see default values.


Workflow level -mgxd

- Pass in subject, parameter in IdentityInterface ([function free nodes](http://miykael.github.io/nipype-beginner-s-guide/firstSteps.html#function-free-nodes))
- Pass subject, parameter in BET node.


Look at different brain extractors optimized for patients.

- [optiBET](https://montilab.psych.ucla.edu/fmri-wiki/optibet/)

# ANTs BrainExtraction on one subject

No workflow.

In [ ]:
from os import path as op

from nipype import DataSink, Node, Workflow
from nipype.interfaces import ants

In [ ]:
template_dir = op.abspath('ants_templates/OASIS-30_Atropos_template/')
brain_template_fname = 'T_template0.nii.gz'
prob_mask_fname = 'T_template0_BrainCerebellumProbabilityMask.nii.gz'

In [ ]:
seg = Node(ants.BrainExtraction(), name='seg')
seg.inputs.anatomical_image = op.abspath('data/case_001_2.nii.gz')
seg.inputs.brain_template = op.join(template_dir, brain_template_fname)
seg.inputs.brain_probability_mask = op.join(template_dir, prob_mask_fname)

# Node to save output files.
sinker = Node(DataSink(), name='sinker')
sinker.inputs.base_directory = op.abspath('ants_bash_test')

In [ ]:

# Workflow.
wf = Workflow(name='ants_bash_seg', base_dir='/om/scratch/Wed/jakubk/')
wf.connect(grab, 't1', seg, 'anatomical_image')
wf.connect(seg, 'BrainSegmentation', sinker, 'segs/.')
wf.connect(seg, 'BrainExtractionMask', sinker, 'extract/.')
wf.connect(seg, 'CorticalThickness', sinker, 'thickness/.')
wf.connect(seg, 'BrainVolumes', sinker, 'vols/.')

# FSL FAST workflow

In [9]:
from os import path as op

from nipype import DataGrabber, DataSink, Node, MapNode, Workflow
from nipype.interfaces import fsl

# Node to grab data.
grab = Node(DataGrabber(outfields=['t1', 'brain']), name='grabber')
grab.inputs.base_directory = '/om/user/jakubk/meningioma/'
grab.inputs.template = '*.nii.gz'
# Change filenames later to specify T1.
grab.inputs.field_template = {'t1': 'data/*.nii.gz',
                              'brain': 'ants_seg_output/brain/*.nii.gz'}
grab.inputs.sort_filelist = True

fast = MapNode(fsl.FAST(), iterfield=['in_files'], name='fast')
fast.inputs.img_type = 1
fast.inputs.probability_maps = True
fast.iterables = ('number_classes', [3, 4, 5])

sinker = Node(DataSink(), name='sinker')
sinker.inputs.base_directory = op.abspath('fast_output')

wf = Workflow(name='fast', base_dir='/om/scratch/Wed/jakubk/')
wf.connect(grab, 't1', fast, 'in_files')
wf.connect(fast, 'probability_maps', sinker, 'prob')
wf.connect(fast, 'restored_image', sinker, 'restored')
wf.connect(fast, 'tissue_class_files', sinker, 'tissue_files')
wf.connect(fast, 'tissue_class_map', sinker, 'tissue_map')

wf.run(plugin='SLURM', plugin_args={'sbatch_args': '--mem=50GB'})

# Garbage

In [ ]:
from glob import glob
import os
import os.path as op

from nipype.interfaces import afni, ants, fsl, slicer, spm, io
import nipype.interfaces.utility as niu
import nipype.pipeline.engine as pe

In [ ]:
PATH = op.abspath('.')
PATH

files = sorted(glob(op.join(PATH, 'data/*.nii.gz')))
print("{} NIFTI files".format(len(files)))

# ANTs BrainExtraction

def get_files():
    return sorted(glob(op.join(PATH, 'data/*.nii.gz')))

anat_files = pe.Node(niu.IdentityInterface(fields=['anatomical']),
                     name='anat_files')
anat_files.iterables = [('anatomical', get_files())]

templates = [op.join(PATH, 
                     'ants_templates/IXI/T_template{}.nii.gz'.format(i)) for i in range(4)]
strip = pe.Node(ants.BrainExtraction(), name='strip')
strip.inputs.dimension = 3
strip.iterables = ('brain_template', templates)
strip.inputs.brain_probability_mask = op.join(PATH, 'ants_templates/IXI/T_templateProbabilityMask.nii.gz')

wf = pe.Workflow(name='ants_brainstripper', base_dir='ants_skullstrip')
wf.connect(anat_files, 'anatomical', strip, 'anatomical_image')
# wf.run(plugin='SLURM')

# BrainExtraction with `mgxd`

In [ ]:
import os.path as op
from nipype import Node, Workflow, DataGrabber, DataSink, MapNode
from nipype.interfaces import ants

base_dir = '/om/user/jakubk/meningioma/'

# Node to grab data.
grab = Node(DataGrabber(outfields=['t1']), name='grabber')
grab.inputs.base_directory = '/om/user/jakubk/meningioma/data'
grab.inputs.template = '*.nii.gz'
# Change filenames later to specify T1.
grab.inputs.field_template = {'t1': '*.nii.gz'}
grab.inputs.sort_filelist = True

# Node to run antsCorticalThickness.
# Segments the anatomical image and should extract brain.
template_dir = op.join(base_dir, 'ants_templates', 'OASIS-30_Atropos_template')
seg = MapNode(ants.CorticalThickness(), iterfield=['anatomical_image'], name='seg')
seg.inputs.dimension = 3
seg.inputs.brain_template = op.join(template_dir, 'T_template0.nii.gz')
seg.inputs.t1_registration_template = op.join(template_dir, 'T_template0_BrainCerebellum.nii.gz')
seg.inputs.brain_probability_mask = op.join(template_dir,
                                            'T_template0_BrainCerebellumProbabilityMask.nii.gz')
seg.inputs.segmentation_priors = [op.join(template_dir, 'Priors2', 'priors{}.nii.gz'.format(i)) for i in range(1, 7)]
seg.inputs.extraction_registration_mask = op.join(template_dir, 'T_template0_BrainCerebellumExtractionMask.nii.gz')
seg.inputs.quick_registration = True
seg.inputs.segmentation_iterations = 1

# Node to save output files.
sinker = Node(DataSink(), name='sinker')
sinker.inputs.base_directory = op.join(base_dir, 'ants_seg_output')

# Workflow.
wf = Workflow(name='ants_seg', base_dir='/om/scratch/Wed/jakubk/')
wf.connect(grab, 't1', seg, 'anatomical_image')
wf.connect(seg, 'BrainSegmentation', sinker, 'segs/.')
wf.connect(seg, 'BrainExtractionMask', sinker, 'extract/.')
wf.connect(seg, 'CorticalThickness', sinker, 'thickness/.')
wf.connect(seg, 'BrainVolumes', sinker, 'vols/.')

### Iterate over IXI templates in 3 subjects

In [ ]:
templates = ['ants_templates/IXI/T_template{}.nii.gz'.format(i) for i in range(4)]

strip = pe.Node(ants.BrainExtraction(), name='strip')
strip.inputs.dimension = 3
strip.iterables = [('brain_template', templates),]
strip.inputs.brain_probability_mask ='ants_templates/IXI/T_templateProbabilityMask.nii.gz'

### No iterable

In [ ]:
strip = ants.BrainExtraction()
strip.inputs.dimension = 3
strip.inputs.anatomical_image ='data/case_001_2.nii.gz'
strip.inputs.brain_template = 'ants_templates/IXI/T_template0.nii.gz'
strip.inputs.brain_probability_mask ='ants_templates/IXI/T_templateProbabilityMask.nii.gz'

In [ ]:
id_node = pe.Node(IdentityInterface(fields=['anatomical']), name='id_node')
id_node.iterables = ('anatomical', files[:1])

templates = ['ants_templates/IXI/T_template{}.nii.gz'.format(i) for i in range(4)]
strip = pe.Node(ants.BrainExtraction(), name='strip')
strip.inputs.dimension = 3
strip.iterables = ('brain_template', templates)
strip.inputs.brain_probability_mask = op.join(PATH, 'ants_templates/IXI/T_templateProbabilityMask.nii.gz')

# datasink = pe.Node(io.DataSink(), name='sink')
# datasink.inputs.base_directory = 'brains'

skullstrip = pe.Workflow(name='skullstrip', base_dir='ants_brain')
skullstrip.connect(id_node, 'anatomical', strip, 'anatomical_image')
# skullstrip.connect(strip, 'out_file', convert, 'in_file')
# skullstrip.connect(strip, 'out_file', datasink, 'container')

In [ ]:
skullstrip.run()

In [ ]:
out = id_node.run()
out.outputs.anatomical

# Trying DataSink

In [ ]:
files[0]

In [ ]:
def 

In [ ]:
id_node = pe.Node(IdentityInterface(fields=['anatomical']), name='id_node')
id_node.iterables = [('anatomical', ['data/case_001_2.nii.gz'])]

bet = pe.Node(fsl.BET(), name='bet')

sink = pe.Node(io.DataSink(), name='sinker')
sink.inputs.base_directory = 'test_sink'

In [ ]:
wf = pe.Workflow(name='test_wf')
wf.connect(id_node, 'anatomical', bet, 'in_file')
wf.connect(bet, 'out_file', sink, 'brain')
wf.run(plugin='MultiProc')

In [ ]:
fsl_data_dir = '/cm/shared/openmind/fsl/5.0.9/data/'
brainextraction = ants.BrainExtraction()
brainextraction.inputs.dimension = 3
brainextraction.inputs.anatomical_image ='data/case_001_2.nii.gz'
brainextraction.inputs.brain_template = op.join(fsl_data_dir, 'standard/MNI152_T1_2mm_brain_mask.nii.gz')
brainextraction.inputs.brain_probability_mask = op.join(fsl_data_dir, 'atlases/MNI/MNI-maxprob-thr50-2mm.nii.gz')
brainextraction.inputs.args = '-o "brain_"'
brainextraction.run()

In [ ]:
help(ants.BrainExtraction)

In [ ]:
a = afni.SkullStrip()
a.inputs.in_file = brains[0]
a.inputs.args = '-use_skull'
a.inputs.out_file = "case_001_"
a.run()

In [ ]:
a = afni.AFNItoNIFTI()
a.inputs.in_file = "case_001_+orig.HEAD"
a.inputs.out_file = "case_001_1_useskull.nii.gz"
a.run()

In [ ]:
files = sorted(glob('data/*.nii.gz'))
print("{} NIFTI files".format(len(files)))

id_node = pe.Node(IdentityInterface(fields=['raw_files']), name='id_node')
id_node.iterables = ('raw_files', files[:5])

strip = pe.Node(afni.SkullStrip(), name='strip')
convert = pe.Node(afni.AFNItoNIFTI(), name='convert')
datasink = pe.Node(io.DataSink(), name='sink')
datasink.inputs.base_directory = 'brains'

skullstrip = pe.Workflow(name='skullstrip')
skullstrip.connect(id_node, 'raw_files', strip, 'in_file')
# skullstrip.connect(strip, 'out_file', convert, 'in_file')
skullstrip.connect(strip, 'out_file', datasink, 'container')

In [ ]:
strip_wf.run(plugin='MultiProc')

In [ ]:
strip_wf.write_graph()

In [ ]:
strip = pe.Workflow(name='strip', base_dir='test-wf')
strip.connect(id_node, 'brain', afni_strip, 'in_file')
# strip.connect(id_node, 'thres', bet, 'frac')
eg = strip.run()

In [ ]:
convert = afni.AFNItoNIFTI()
convert.inputs.in_file = 'afni_+orig.HEAD'
convert.inputs.out_file = 'afni_+orig.nii.gz'
convert.run()

In [ ]:
help(afni.SkullStrip)

In [ ]:
# ANTS brain extraction
# 1. create template using buildtemplateparallel.sh
# 2. warp brain masks from LPBA40 data to the template space
#    to create probabilistic mask of the brain in template 
#
# ants.BrainExtraction() requires `anatomical_image`,
# `brain_probability_mask`, and `brain_template`.

In [ ]:
# # AFNI (add afni to environment)
# afni_strip = pe.Node(afni.SkullStrip(), name='afni_strip')

# ANTS
afni_strip = pe.Node(afni.SkullStrip(), name='afni_strip')

In [ ]:
afni_strip = afni.SkullStrip()
afni_strip.inputs.in_file = brains[0]
afni_strip.inputs.out_file = "afni_"
# afni_strip.inputs.args = '-use_skull'
afni_strip.run()